In [ ]:
### Only run once at the very first time
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install fschat

### Restart Jupyter Kernel / Colab Runtime

In [ ]:
### Import packages

import pandas as pd
import numpy as np
import argparse
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from fastchat.model import load_model, get_conversation_template, add_model_args

directory='/content' ### Set Directory

In [ ]:
%%bash
# Script to make sure data files can be used

directory='/content' ### Must be the same as the variable directory

if [ ! -d $directory ]; then
mkdir -p $directory
fi

cd $directory
wget https://storage.googleapis.com/yantk-siads-699/699_final.tar.gz
tar -xvzf ${directory}/699_final.tar.gz .
mkdir ${directory}/699
mkdir ${directory}/699/699
mv ${directory}/Data\ 20230612 ${directory}/699/699
mv ${directory}/model ${directory}/699/699

In [ ]:
### Read testing set

test_df = pd.read_csv(directory + '/699/699/Data 20230612/Youtube/test_set_youtube.csv',  lineterminator='\n')

In [ ]:
test_df.loc[1,'description']

'Go to ecb.co.uk to join We Are England Cricket Supporters for free and get priority access to tickets and much more!Watch match highlights from Day 4 of the 3rd LV= Insurance Test between England and New Zealand at Headingley Stadium.'

In [ ]:
test_df.loc[1,'title']

'Pope & Root Masterclass! | Highlights | England v New Zealand - Day 4 | 3rd LV= Insurance Test 2022'

In [ ]:
test_df["description"] = test_df["description"].astype(str)
test_df["title"] = test_df["title"].astype(str)
test_df["combined"] = test_df["title"] + '' + test_df["description"]
test_df.sample(5)

,Unnamed: 0,video_id,title,description,en_tag_list,top_tags,combined,vicuna
434,722,3vPJ-12LxrM,Panda,Start listening with a 30-day Audible trial. G...,"['domics', 'animation']","['domics', 'animation']",PandaStart listening with a 30-day Audible tri...,"['panda', 'sober', 'conversation', 'twitch', '..."
1805,9176,EobMnSpjfTU,Inside Troye Sivan's Victorian-Era Melbourne H...,Today Architectural Digest brings you to Austr...,"['ad', 'ad celebrity home tour', 'ad home tour...","['ad', 'ad celebrity home tour', 'ad home tour...",Inside Troye Sivan's Victorian-Era Melbourne H...,"['Troye Sivan', 'Melbourne', 'Victorian-Era', ..."
2197,61025,V6zGk373zuo,How to get The Foundation Skin Early!,Here's how to get The Foundation skin (aka The...,"['the foundation fortnite', 'where is foundati...","['fortnite storyline', 'fortnite shorts', 'dag...",How to get The Foundation Skin Early!Here's ho...,"['The Foundation Skin', 'Fortnite', 'Chapter 3..."
2597,20793,9tA1-IiCTE4,I Trolled Him With KLOMBO Boss - Chapter 3,I Trolled Him With KLOMBO Boss In Fortnite Cha...,"['fortnite', 'klombo fortnite', 'fortnite chap...","['fortnite', 'klombo fortnite', 'fortnite chap...",I Trolled Him With KLOMBO Boss - Chapter 3I Tr...,"['Fortnite', 'Trolling', 'Klombo', 'Butter Cak..."
3979,12219,KWB8sa34cl8,England cruises by Ukraine to advance to the E...,Watch extended highlights of England's dominan...,"['harry kane', 'kane', 'england', 'ukraine', '...","['harry kane', 'kane', 'england', 'ukraine', '...",England cruises by Ukraine to advance to the E...,"['soccer', 'football', 'Euro 2020', 'England',..."


In [ ]:
### Load Vicuna model (ref: https://github.com/lm-sys/FastChat/blob/main/fastchat/serve/huggingface_api.py)

model_path = 'lmsys/vicuna-13b-v1.3'

model, tokenizer = load_model(model_path, 'cuda', 1, load_8bit=True)

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

100%|██████████| 3/3 [02:05<00:00, 41.72s/it]


In [ ]:
### function to call Vicuna (ref: https://github.com/lm-sys/FastChat/blob/main/fastchat/serve/huggingface_api.py)
### Define a function for prompt engineering
### Create prompts to instruct Vicuna 13B to follow this pattern

def vicuna_predict(combined):

  example_output = """['first we feast', 'fwf', ..., 'bartender']"""
  msg = f"""You are a tags generator for a video.
  Example output format: {example_output}
  Generate 10 tags for a video:
  Video Description: {combined}
  Output:
  """

  conv = get_conversation_template(model_path)
  conv.append_message(conv.roles[0], msg)
  conv.append_message(conv.roles[1], None)
  prompt = conv.get_prompt()

  input_ids = tokenizer([prompt]).input_ids
  output_ids = model.generate(
      torch.as_tensor(input_ids).cuda(),
      do_sample=True,
      temperature=0.4,   ### fine tuned after qualitative exploration
      repetition_penalty=1.0, ### fine tuned after qualitative exploration
      max_new_tokens=500,
  )

  output_ids = output_ids[0][len(input_ids[0]) :]
  outputs = tokenizer.decode(
      output_ids, skip_special_tokens=True, spaces_between_special_tokens=False
  )

  #print(f"{conv.roles[0]}: {msg}")
  #print(f"{conv.roles[1]}: {outputs}")
  return outputs

In [ ]:
%%time
### Simple Test
vicuna_predict('Why was I on TV in India?!SUBSCRIBE TO @No Context JackSucksAtLife Why was I on TV in India?! Join the subreddit: https://reddit.com/r/jacksucksatlife/ Subscribe with notifications 🔔 and check out my other Youtube channels below! @JackSucksAtLife @JackSucksAtStuff @JackSucksAtGeography @Jack Massey Welsh @JackSucksAtClips @No Context JackSucksAtLife twitter: https://twitter.com/jackmasseywelshinstagram: https://instagram.com/jackmasseywelshmy subreddit: https://reddit.com/r/jacksucksatlife/tiktok: https://www.tiktok.com/@jackmasseywvideo epically edited by: https://twitter.com/kairossbestbusiness email: jack @ skycade . netaffiliate links:minecraft skinpack: https://www.minecraft.net/en-us/marketplace/pdp?id=8f8a099e-2c4d-4bc4-b552-f14408bebc82limited edition youtooz - https://youtooz.com/collections/jacksucksatlifesecretlab gaming chair - http://bit.ly/3nmYnexsubscribe animations - https://creatorset.com/?ref=jackJackSucksAtLife, also known as Jack Massey Welsh started his Youtube channel playing Minecraft and making funny Minecraft videos. More recently he is known for his Youtube Play Button & Youtube Awards collection. He is the first Youtuber to get a Diamond Play Button without 10 million subscribers. Jack Massey Welsh holds a Guinness World Record for how many Youtube Channels he')

CPU times: user 21.3 s, sys: 2.59 s, total: 23.9 s
Wall time: 23.8 s


"['jackmasseywelsh', 'jacksucksatlife', 'jacksucksatstuff', 'jacksucksatgeography', 'jackmasseywelsh', 'jacksucksatclips', 'nocontextjacksucksatlife', 'tiktok', 'kairossbestbusiness', 'minecraft', 'youtooz', 'gamingchair', 'creatorset', 'guinnessworldrecord', 'diamondplaybutton', 'youtubeawards', 'youtubeplaybutton']"

In [ ]:
tmp_df = test_df.sample(5).copy()
tmp_df

,Unnamed: 0,video_id,title,description,en_tag_list,top_tags,combined
2266,44111,41wizkK5jHY,RAMOS Wants UNITED TRANSFER? Man Utd Transfer ...,Man Utd could be offered a Sergio Ramos transf...,"['manchester united', 'man united', 'man utd',...","['manchester united', 'man united', 'man utd',...",RAMOS Wants UNITED TRANSFER? Man Utd Transfer ...
1133,142,gSc0o6xVnmY,Juice WRLD & The Weeknd - Smile (Official Lyri...,Juice WRLD & The Weeknd - Smile is available n...,"['juicewrld', 'juiceworld', 'juice world', 'ju...","['juicewrld', 'juiceworld', 'juice world', 'ju...",Juice WRLD & The Weeknd - Smile (Official Lyri...
1209,54697,qbei6-vtl0Q,YENA (최예나) - Love War (Feat. BE'O) MV Teaser 1,YENA (최예나) - Love War (Feat. BE'O) MV Teaser 1...,"['k-culture', 'korean music', 'mv', 'music vid...","['k-culture', 'korean music', 'mv', 'music vid...",YENA (최예나) - Love War (Feat. BE'O) MV Teaser 1...
5256,45458,eWZeD5uVxsU,HIGHLIGHTS | Norwich City 7-0 Huddersfield Town,All of the action from Carrow Road as City put...,"['football', 'soccer', 'norwich', 'norwich cit...","['football', 'soccer', 'norwich', 'norwich cit...",HIGHLIGHTS | Norwich City 7-0 Huddersfield Tow...
2633,44943,Ki3hP3SUy60,MANCHESTER UNITED vs AC MILAN With Mark GOLDBR...,Manchester United vs AC Milan Match Reaction h...,"['manchester united', 'man united', 'man utd',...","['manchester united', 'man united', 'man utd',...",MANCHESTER UNITED vs AC MILAN With Mark GOLDBR...


In [ ]:
%%time
### Bulk Tags generation
tmp_df['vicuna'] = tmp_df['combined'].apply(vicuna_predict)

CPU times: user 47.2 s, sys: 4.89 s, total: 52.1 s
Wall time: 52 s


In [ ]:
tmp_df[['top_tags','vicuna', 'combined']]

,top_tags,vicuna,combined
2266,"['manchester united', 'man united', 'man utd',...","['ramos', 'transfer', 'man utd', 'centre back'...",RAMOS Wants UNITED TRANSFER? Man Utd Transfer ...
1133,"['juicewrld', 'juiceworld', 'juice world', 'ju...","['juice wrld', 'the weeknd', 'smile', 'lyric v...",Juice WRLD & The Weeknd - Smile (Official Lyri...
1209,"['k-culture', 'korean music', 'mv', 'music vid...","['K-pop', 'MV', 'YENA', 'BE'O', 'Love War', 'a...",YENA (최예나) - Love War (Feat. BE'O) MV Teaser 1...
5256,"['football', 'soccer', 'norwich', 'norwich cit...","['Norwich City', 'Huddersfield Town', 'EFL Cha...",HIGHLIGHTS | Norwich City 7-0 Huddersfield Tow...
2633,"['manchester united', 'man united', 'man utd',...","['manchester united', 'ac milan', 'europa leag...",MANCHESTER UNITED vs AC MILAN With Mark GOLDBR...


In [ ]:
%%time

test_df['vicuna'] = test_df['combined'].apply(vicuna_predict)

test_df.to_csv(directory + '/699/699/Data 20230612/Youtube/vicuna_results.csv')
test_df.to_csv(directory + '/699/699/Data 20230612/Youtube/vicuna_results.csv.backup')
backup_df = test_df.copy()

Token indices sequence length is longer than the specified maximum sequence length for this model (2729 > 2048). Running this sequence through the model will result in indexing errors


CPU times: user 18h 13min 45s, sys: 1h 53min 5s, total: 20h 6min 50s
Wall time: 20h 4min 35s


In [ ]:
test_df[['vicuna']]

,vicuna
0,"['love', 'Kid Cudi', 'The Boy Who Flew To The ..."
1,"['cricket', 'test match', 'england cricket', '..."
2,"['sypherpk', 'fortnite', 'battle royale', 'gam..."
3,"['basketball', 'dunk', 'sports', 'viral', 'ins..."
4,"['The Arnold Sisters', 'pregnancy announcement..."
...,...
5494,"['politics', 'interview', 'french president', ..."
5495,"['Soolking', 'Lynda', 'Heuss', 'L\'Algérino', ..."
5496,"['RNC', 'Republican National Convention', 'Tru..."
5497,"['barstool sports', 'grilling competition', 'r..."


Check Broken Predictions or Results not in required format

In [ ]:
test_df[~test_df['vicuna'].str.get(0).isin(['['])][['vicuna']]

,vicuna
58,1. Taliban\n2. Afghanistan\n3. Kabul\n4. Milit...
108,1. Twitter\n2. Donald Trump\n3. Suspension\n4....
110,1. #TooGoodToGo #RestaurantLeftovers #FoodWast...
132,1. 2021 pandemic\n2. Posh lifestyle\n3. Luxury...
133,1. firstmarriedcouple\n2. firstmarriedtrip\n3....
...,...
5354,1. Suni Lee\n2. Gold Medal\n3. Women's Gymnast...
5373,1. New Year's Eve celebrations\n2. Sydney New ...
5379,1. POV time limit challenge\n2. Time managemen...
5412,1. Bugzy Malone\n2. Salvador\n3. Music Video\n...


In [ ]:
test_df[~test_df['vicuna'].str.get(-1).isin([']'])][['vicuna']]

,vicuna
41,"['roblox', 'metaverse', 'gaming', 'virtual rea..."
58,1. Taliban\n2. Afghanistan\n3. Kabul\n4. Milit...
108,1. Twitter\n2. Donald Trump\n3. Suspension\n4....
110,1. #TooGoodToGo #RestaurantLeftovers #FoodWast...
132,1. 2021 pandemic\n2. Posh lifestyle\n3. Luxury...
...,...
5387,"['Snickers', 'US vs UK', 'Food Wars', 'Food', ..."
5412,1. Bugzy Malone\n2. Salvador\n3. Music Video\n...
5462,1. Mukbang\n2. KSI\n3. JJ\n4. Food\n5. Eating\...
5463,"['kenny omega', 'ae"
